In [1]:
# magic! (don't worry about this)
%load_ext autoreload
%autoreload 2
#%matplotlib inline

In [228]:
# let us import some useful things
from lib import *
from classifiers import *
from numpy import mean
#from graphs import *
from sklearn import tree
import numpy as np


In [229]:
# load the data same as before
microarray_file_name = '../data/leukemia_ALL_AML_matrix.txt'
labels_file_name = '../data/leukemia_ALL_AML_labels.txt'
data_store = DataSet(microarray_file_name, labels_file_name)

In [230]:
# This train function is *almost* the same as the train method that 
# your knn classifier. The difference is that this is a function that
# takes as one of the parameters the decision tree, so it will be called
# like: train(decision_tree, train_samples)
def train(decision_tree, train_samples):
    feature_array = [sample.get_gene_profile() for sample in train_samples]
    labels = [sample.get_label() for sample in train_samples]
    decision_tree.fit(feature_array, labels)

def classify(decision_tree, test_samples):
    labelled_samples = []
    feature_array = [sample.get_gene_profile() for sample in test_samples]
    results = decision_tree.predict(feature_array)
    labelled_samples = [(test_samples[i], results[i]) for i in range(len(test_samples))]
    return labelled_samples


In [231]:
# Here, you can set criterion = "entropy" or "gini", which will determine
# what equation the decision tree will use to measure the quality of a split
#
# You can set max_features=None, "sqrt", or "log2", which will determine how many
# features the decision tree will use. Setting it to None will use all the features,
# sqrt will use sqrt(number of features) and log2 will use log2(number of features)
#
# Play around! What settings are best? Do they change for the different data sets?
decision_tree = tree.DecisionTreeClassifier(criterion="entropy", max_features=None, random_state=408)
train(decision_tree, data_store.get_train_set())

classified_samples = classify(decision_tree, data_store.get_test_set())
# let's evaluate how well the classifier worked
evaluate_results(classified_samples)

Accuracy: 96.00%


In [ ]:
#plot_errors(classified_samples)

In [269]:
def bootstrap(data):
    N = len(data)
    sample = np.random.choice(N, N)
    return [data[i] for i in sample]

def bagTrain(ntrees):
    myTrees = []
    for i in range(ntrees):
        decision_tree = tree.DecisionTreeClassifier(criterion="entropy", max_features=None)
        train(decision_tree, bootstrap(data_store.get_train_set()))
        myTrees.append(decision_tree)
    return myTrees

def bagClassify(myTrees, test_samples):
    classification = []
    for sample in test_samples:
#         print sample.get_gene_profile()
        judgment = []
        for tree in myTrees:
            judgment.append(classify(tree, [sample]))
        judgment = [item[0][1] for item in judgment]
        classification.append([sample, mean(judgment) > 0.5])
    return classification
    


In [287]:
myTrees = bagTrain(1)
classes = bagClassify(myTrees, data_store.get_test_set())
evaluate_results(classes)

Accuracy: 92.00%


Run the regular tree and the bagged tree multiple times. What do you notice? 